# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
J. Liu  ->  J. Liu  |  ['J. Liu']
J. Eberhardt  ->  J. Eberhardt  |  ['J. Eberhardt']
T. Henning  ->  T. Henning  |  ['T. Henning']
T. Trifonov  ->  T. Trifonov  |  ['T. Trifonov']
B. Maity  ->  B. Maity  |  ['B. Maity']


K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
M. Schirmer  ->  M. Schirmer  |  ['M. Schirmer']
J. Liu  ->  J. Liu  |  ['J. Liu']
Arxiv has 66 new papers today
          9 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/9 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2408.05275


extracting tarball to tmp_2408.05275...

 done.


Found 121 bibliographic references in tmp_2408.05275/AandA.bbl.
syntax error in line 536: '=' expected
Retrieving document from  https://arxiv.org/e-print/2408.05292


...

extracting tarball to tmp_2408.06215... done.
Retrieving document from  https://arxiv.org/e-print/2408.06217


extracting tarball to tmp_2408.06217...

 done.
Retrieving document from  https://arxiv.org/e-print/2408.06338


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/tmp/ipykernel_2728/1211882699.py:51: LatexWarning: 2408.06217 did not run properly
'PosixPath' object is not subscriptable
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


extracting tarball to tmp_2408.06338... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.05275-b31b1b.svg)](https://arxiv.org/abs/2408.05275) | **Euclid preparation. The Cosmic Dawn Survey (DAWN) of the Euclid Deep and Auxiliary Fields**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2024-08-13*|
|*Comments*| *16 pages, 10 figures, submitted to A&A*|
|**Abstract**|            Euclid will provide deep NIR imaging to $\sim$26.5 AB magnitude over $\sim$59 deg$^2$ in its deep and auxiliary fields. The Cosmic DAWN survey complements the deep Euclid data with matched depth multiwavelength imaging and spectroscopy in the UV--IR to provide consistently processed Euclid selected photometric catalogs, accurate photometric redshifts, and measurements of galaxy properties to a redshift of $z\sim 10$. In this paper, we present an overview of the survey, including the footprints of the survey fields, the existing and planned observations, and the primary science goals for the combined data set.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.05296-b31b1b.svg)](https://arxiv.org/abs/2408.05296) | **Euclid preparation. Cosmic Dawn Survey: Data release 1 multiwavelength catalogues for Euclid Deep Field North and Euclid Deep Field Fornax**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2024-08-13*|
|*Comments*| **|
|**Abstract**|            The Cosmic Dawn Survey (DAWN survey) provides multiwavelength (UV/optical to mid-IR) data across the combined 59 deg$^{2}$ of the Euclid Deep and Auxiliary fields (EDFs and EAFs). Here, the first public data release (DR1) from the DAWN survey is presented. DR1 catalogues are made available for a subset of the full DAWN survey that consists of two Euclid Deep fields: Euclid Deep Field North (EDF-N) and Euclid Deep Field Fornax (EDF-F). The DAWN survey DR1 catalogues do not include $Euclid$ data as they are not yet public for these fields. Nonetheless, each field has been covered by the ongoing Hawaii Twenty Square Degree Survey (H20), which includes imaging from CFHT MegaCam in the new $u$ filter and from Subaru Hyper Suprime-Cam (HSC) in the $griz$ filters. Each field is further covered by $Spitzer$/IRAC 3.6-4.5$\mu$m imaging spanning 10 deg$^{2}$ and reaching $\sim$25 mag AB (5$\sigma$). All present H20 imaging and all publicly available imaging from the aforementioned facilities are combined with the deep $Spitzer$/IRAC data to create source catalogues spanning a total area of 16.87 deg$^{2}$ in EDF-N and 2.85 deg$^{2}$ in EDF-F for this first release. Photometry is measured using The Farmer, a well-validated model-based photometry code. Photometric redshifts and stellar masses are computed using two independent codes for modeling spectral energy distributions: EAZY and LePhare. Photometric redshifts show good agreement with spectroscopic redshifts ($\sigma_{\rm NMAD} \sim 0.5, \eta < 8\%$ at $i < 25$). Number counts, photometric redshifts, and stellar masses are further validated in comparison to the COSMOS2020 catalogue. The DAWN survey DR1 catalogues are designed to be of immediate use in these two EDFs and will be continuously updated. Future data releases will provide catalogues of all EDFs and EAFs and include $Euclid$ data.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.05612-b31b1b.svg)](https://arxiv.org/abs/2408.05612) | **Mass determination of two Jupiter-sized planets orbiting slightly evolved stars: TOI-2420 b and TOI-2485 b**  |
|| I. Carleo, et al. -- incl., <mark>J. Eberhardt</mark>, <mark>T. Henning</mark>, <mark>T. Trifonov</mark> |
|*Appeared on*| *2024-08-13*|
|*Comments*| **|
|**Abstract**|            Hot and warm Jupiters might have undergone the same formation and evolution path, but the two populations exhibit different distributions of orbital parameters, challenging our understanding on their actual origin. The present work, which is the results of our warm Jupiters survey carried out with the CHIRON spectrograph within the KESPRINT collaboration, aims to address this challenge by studying two planets that could help bridge the gap between the two populations. We report the confirmation and mass determination of a hot Jupiter (orbital period shorter than 10 days), TOI-2420\,b, and a warm Jupiter, TOI-2485\,b. We performed a joint analysis using a wide variety of spectral and photometric data in order to characterize these planetary systems. We found that TOI-2420\,b has an orbital period of P$_{\rm b}$=5.8 days, a mass of M$_{\rm b}$=0.9 M$_{\rm J}$ and a radius of R$_{\rm b}$=1.3 R$_{\rm J}$, with a planetary density of 0.477 \gc; while TOI-2485\,b has an orbital period of P$_{\rm b}$=11.2 days, a mass of M$_{\rm b}$=2.4 M$_{\rm J}$ and a radius of R$_{\rm b}$=1.1 R$_{\rm J}$ with density 2.36 \gc. With current parameters, the migration history for TOI-2420\,b and TOI-2485\,b is unclear: the high-eccentricity migration scenarios cannot be ruled out, and TOI-2485\,b's characteristics may rather support this scenario.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.05722-b31b1b.svg)](https://arxiv.org/abs/2408.05722) | **Prospects of FRBs and Large Scale 21 cm Power Spectra in Constraining the Epoch of Reionization**  |
|| <mark>B. Maity</mark> |
|*Appeared on*| *2024-08-13*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**|            The Epoch of Reionization (EoR) is a crucial link to grasp the complete evolutionary history of the universe. Several attempts with a variety of observables have been utilized in the past to understand the thermal and ionization evolution of the Intergalactic Medium during EoR. In this study, we explore the simultaneous prospects of two important observables which are expected to be available in the near future, i.e. Dispersion Measure (DM) of high redshift FRBs and large scale 21 cm power spectra. For this purpose, we use an earlier developed explicitly photon conserving semi numerical model, $\texttt{SCRIPT}$ including realistic recombination and radiative feedback effect. We check that DM evolution of 100 mock FRBs at high redshifts ($7.0\le z\le15.0$) is sufficient to recover the underlying reionization model, while 1000 FRB mocks at redshift range can constrain the reionization timeline within the percentage level uncertainties at 68\% confidence limit. Further, we study the effect of including large scale 21~cm power spectra (using only a single bin, $k\sim0.14~h/\mathrm{cMpc}$) at three redshifts along with FRB DM distribution. The joint exploration using these two observables can significantly improve the constraints on the various parameters ($\lesssim 8\%$ uncertainties for reionization interval and midpoint at 95\% confidence) alleviating the degeneracies and can narrow down the thermal history of the universe by discarding some of the extreme heating models.         |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.06215-b31b1b.svg)](https://arxiv.org/abs/2408.06215) | **Gas-phase Fe/O and Fe/N abundances in Star-Forming Regions. Relations between nucleosynthesis, metallicity and dust**  |
|| J. E. Méndez-Delgado, et al. -- incl., <mark>K. Kreckel</mark> |
|*Appeared on*| *2024-08-13*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**|            In stars, metallicity is usually traced using Fe, while in nebulae, O serves as the preferred proxy. Both elements have different nucleosynthetic origins and are not directly comparable. Additionally, in ionized nebulae, Fe is heavily depleted onto dust grains. We investigate the distribution of Fe gas abundances in a sample of 452 star-forming nebulae with \feiii~$\lambda 4658$ detections and their relationship with O and N. Additionally, we analyze the depletion of Fe onto dust grains in photoionized environments. We homogeneously determine the chemical abundances with direct determinations of electron temperature ($T_e$), considering the effect of possible internal variations of this parameter. We adopt a sample of 300 Galactic stars to interpret the nebular findings. We find a moderate linear correlation ($r=-0.59$) between Fe/O and O/H. In turn, we report a stronger correlation ($r=-0.80$) between Fe/N and N/H. We interpret the tighter correlation as evidence of Fe and N being produced on similar timescales while Fe-dust depletion scales with the Fe availability. The apparently flat distribution between Fe/N and N/H in Milky Way stars supports this interpretation. We find that when 12+log(O/H)<7.6, the nebulae seem to reach a plateau value around $\text{log(Fe/O)} \approx -1.7$. If this trend is confirmed, it would be consistent with a very small amount of Fe-dust in these systems, similar to what is observed in high-z galaxies discovered by the James Webb Space Telescope (JWST). We derive a relationship that allows us to approximate the fraction of Fe trapped into dust in ionized nebulae. If the O-dust scales in the same way, its possible contribution in low metallicity nebulae would be negligible. After analyzing the Fe/O abundances in J0811+4730 and J1631+4426, we do not see evidence of the presence of very massive stars with $M_\text{init}>300M_{\odot}$ in these systems.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.05381-b31b1b.svg)](https://arxiv.org/abs/2408.05381) | **Statistics of Solar White-Light Flares I: Optimization of Identification Methods and Application**  |
|| Y. Cai, Y. Hou, T. Li, <mark>J. Liu</mark> |
|*Appeared on*| *2024-08-13*|
|*Comments*| *Comments welcome. 18 pages, 9 figures, 1 table*|
|**Abstract**|            White-light flares (WLFs) are energetic activity in stellar atmosphere. However, the observed solar WLF is relatively rare compared to stellar WLFs or solar flares observed at other wavelengths, limiting our further understanding solar/stellar WLFs through statistical studies. By analyzing flare observations from the \emph{Solar Dynamics Observatory (SDO)}, here we improve WLF identification methods for obtaining more solar WLFs and their accurate light curves from two aspects: 1) imposing constraints defined by the typical temporal and spatial distribution characteristics of WLF-induced signals; 2) setting the intrinsic threshold for each pixel in the flare ribbon region according to its inherent background fluctuation rather than a fixed threshold for the whole region. Applying the optimized method to 90 flares (30 C-class ones, 30 M-class ones, and 30 X-class ones) for a statistical study, we identified a total of 9 C-class WLFs, 18 M-class WLFs, and 28 X-class WLFs. The WLF identification rate of C-class flares reported here reaches 30\%, which is the highest to date to our best knowledge. It is also revealed that in each GOES energy level, the proportion of WLFs is higher in confined flares than that in eruptive flares. Moreover, a power-law relation is found between the WLF energy (\emph{E}) and duration ($\tau$): $\tau \propto {E}^{0.22}$, similar to those of solar hard/soft X-ray flares and other stellar WLFs. These results indicate that we could recognize more solar WLFs through optimizing the identification method, which will lay a base for future statistical and comparison study of solar and stellar WLFs.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.06338-b31b1b.svg)](https://arxiv.org/abs/2408.06338) | **Closeby Habitable Exoplanet Survey (CHES). II. An Observation Strategy for the Target Stars**  |
|| D. Tan, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2024-08-13*|
|*Comments*| *20 pages, 12 figures, accepted for publication in AJ*|
|**Abstract**|            The Closeby Habitable Exoplanet Survey (CHES) constitutes a mission intricately designed to systematically survey approximately 100 solar-type stars located within the immediate proximity of the solar system, specifically within a range of 10 parsecs. The core objective of this mission is the detection and characterization of potentially habitable Earth-like planets or super-Earths within the habitable zone of these stars. The CHES mission obtains high-precision astrometric measurements of planets orbiting the target stars by observing angular distance variations between the target star and reference stars. As a result, we surveyed the relevant parameters of both target and reference stars in detail, conducting a thorough analysis and calculation of the required observation accuracy, the number of observations, and the priority assigned to each target star. Observational emphasis will be concentrated on targets considered of higher priority, ensuring the effectiveness of their observation capabilities. Through this approach, we formulate a five-year observation strategy that will cover all the target stars within a six-month timeframe. The strategy not only fulfills the required observing capability but also exhibit high efficiency simultaneously, providing an executable program for future mission. Over the span of the mission's five-year duration, a cumulative observation time of 29,220 hours will be available. Approximately 86 percent of this, totaling 25,120 hours, is allocated for the observation of target stars. This allocation leaves approximately 4,100 hours for extended scientific observation programs. We have also performed simulated observations based on this strategy and verified its observational capability for exoplanets.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.05292-b31b1b.svg)](https://arxiv.org/abs/2408.05292) | **Large-scale cosmic ray anisotropies with 19 years of data from the Pierre Auger Observatory**  |
|| P. A. Collaboration, et al. |
|*Appeared on*| *2024-08-13*|
|*Comments*| **|
|**Abstract**|            Results are presented for the measurement of large-scale anisotropies in the arrival directions of ultra-high-energy cosmic rays detected at the Pierre Auger Observatory during 19 years of operation, prior to AugerPrime, the upgrade of the Observatory. The 3D dipole amplitude and direction are reconstructed above $4\,$EeV in four energy bins. Besides the established dipolar anisotropy in right ascension above $8\,$EeV, the Fourier amplitude of the $8$ to $16\,$EeV energy bin is now also above the $5\sigma$ discovery level. No time variation of the dipole moment above $8\,$EeV is found, setting an upper limit to the rate of change of such variations of $0.3\%$ per year at the $95\%$ confidence level. Additionally, the results for the angular power spectrum are shown, demonstrating no other statistically significant multipoles. The results for the equatorial dipole component down to $0.03\,$EeV are presented, using for the first time a data set obtained with a trigger that has been optimized for lower energies. Finally, model predictions are discussed and compared with observations, based on two source emission scenarios obtained in the combined fit of spectrum and composition above $0.6\,$EeV.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'PosixPath' object is not subscriptable</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.06217-b31b1b.svg)](https://arxiv.org/abs/2408.06217) | **Euclid: The Early Release Observations Lens Search Experiment**  |
|| J. A. A. Barroso, et al. -- incl., <mark>K. Jahnke</mark>, <mark>M. Schirmer</mark> |
|*Appeared on*| *2024-08-13*|
|*Comments*| *21 pages, 20 figures, submitted to A&A*|
|**Abstract**|            We investigate the ability of the Euclid telescope to detect galaxy-scale gravitational lenses. To do so, we perform a systematic visual inspection of the $0.7\,\rm{deg}^2$ Euclid ERO data towards the Perseus cluster using both the high-resolution VIS $I_{\scriptscriptstyle\rm E}$ band, and the lower resolution NISP bands. We inspect every extended source brighter than magnitude $23$ in $I_{\scriptscriptstyle\rm E}$ with $41$ expert human classifiers. This amounts to $12\,086$ stamps of $10^{\prime\prime}\,\times\,10^{\prime\prime}$. We find $3$ grade A and $13$ grade B candidates. We assess the validity of these $16$ candidates by modelling them and checking that they are consistent with a single source lensed by a plausible mass distribution. Five of the candidates pass this check, five others are rejected by the modelling and six are inconclusive. Extrapolating from the five successfully modelled candidates, we infer that the full $14\,000\,{\rm deg}^2$ of the Euclid Wide Survey should contain $100\,000^{+70\,000}_{-30\,000}$ galaxy-galaxy lenses that are both discoverable through visual inspection and have valid lens models. This is consistent with theoretical forecasts of $170\,000$ discoverable galaxy-galaxy lenses in Euclid. Our five modelled lenses have Einstein radii in the range $0.\!\!^{\prime\prime}68\,<\,\theta_\mathrm{E}\,<1.\!\!^{\prime\prime}24$, but their Einstein radius distribution is on the higher side when compared to theoretical forecasts. This suggests that our methodology is likely missing small Einstein radius systems. Whilst it is implausible to visually inspect the full Euclid data set, our results corroborate the promise that Euclid will ultimately deliver a sample of around $10^5$ galaxy-scale lenses.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'PosixPath' object is not subscriptable</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2408.05275/./figures/h20_fields.png', 'tmp_2408.05275/./figures/edfs_footprints.png', 'tmp_2408.05275/./figures/uvlf.png', 'tmp_2408.05275/./figures/EAF_footprints.png']
copying  tmp_2408.05275/./figures/h20_fields.png to _build/html/
copying  tmp_2408.05275/./figures/edfs_footprints.png to _build/html/
copying  tmp_2408.05275/./figures/uvlf.png to _build/html/
copying  tmp_2408.05275/./figures/EAF_footprints.png to _build/html/
exported in  _build/html/2408.05275.md
    + _build/html/tmp_2408.05275/./figures/h20_fields.png
    + _build/html/tmp_2408.05275/./figures/edfs_footprints.png
    + _build/html/tmp_2408.05275/./figures/uvlf.png
    + _build/html/tmp_2408.05275/./figures/EAF_footprints.png
found figures ['tmp_2408.05296/./figures/EDFN+F_mag_magErr.png', 'tmp_2408.05296/./figures/fields_layout_in.png', 'tmp_2408.05296/./figures/EDFN_depthmaps_hex.png', 'tmp_2408.05296/./figures/EDFF_depthmaps_hex.png']
copying  tmp_2408.05296/./figures/EDFN+F_mag_magErr.png t

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\papernumber}$
$\newcommand{\orcid}[1]$</div>



<div id="title">

# $\Euclid$ Preparation. $\papernumber$ The Cosmic Dawn Survey (DAWN) of the Euclid Deep and Auxiliary Fields

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2408.05275-b31b1b.svg)](https://arxiv.org/abs/2408.05275)<mark>Appeared on: 2024-08-13</mark> -  _16 pages, 10 figures, submitted to A&A_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** $\Euclid$ will provide deep NIR imaging to $\sim$ 26.5 AB magnitude over $\sim$ 59 deg $^2$ in its deep and auxiliary fields. The Cosmic DAWN survey complements the deep $\Euclid$ data with matched depth multiwavelength imaging and spectroscopy in the UV--IR to provide consistently processed $\Euclid$ selected photometric catalogs, accurate photometric redshifts, and measurements of galaxy properties to a redshift of $z\sim 10$ . In this paper, we present an overview of the survey, including the footprints of the survey fields, the existing and planned observations, and the primary science goals for the combined data set.

</div>

<div id="div_fig1">

<img src="tmp_2408.05275/./figures/h20_fields.png" alt="Fig9.1" width="50%"/><img src="tmp_2408.05275/./figures/edfs_footprints.png" alt="Fig9.2" width="50%"/>

**Figure 9. -** Footprints of the DAWN survey image data in the Euclid Deep Fields. Note that the Rubin/LSST footprint in EDF-S shows the expected pointing pattern for the EDF-S Deep Drilling Field and may change once the survey begins \citep{ivezic2019}. (*fig:deep_fields*)

</div>
<div id="div_fig2">

<img src="tmp_2408.05275/./figures/uvlf.png" alt="Fig7" width="100%"/>

**Figure 7. -** Measurements of the ultraviolet luminosity function $\phi$ at $z = \textrm{8--10}$ from the literature \citep{Oesch2018,Bowler2020,Bouwens2021,Finkelstein2022,Adams2023,Donnan2023}. The red and blue lines show extrapolations to the volume of the DAWN survey based on the best fitting Schechter and double power-law (DPL) functions, with the shaded regions showing associated Poisson uncertainties for the DAWN survey volume. Upper limits based on survey volume are indicated with the horizontal lines. The DAWN volume in each panel assumes an area of 59 deg$^2$, a redshift slice of $\pm0.5$ from the redshift bin center, and a \citet{Planck2018}$\Lambda$CDM cosmology. (*fig:uvlf*)

</div>
<div id="div_fig3">

<img src="tmp_2408.05275/./figures/EAF_footprints.png" alt="Fig10" width="100%"/>

**Figure 10. -** Footprints of the DAWN survey image data in the EAFs. The coloured lines show the extent of the image data indicated in the legend. Note that the EAF footprints are approximations based on the \Euclid survey plan. Updated footprints that to reflect the actual \Euclid coverage in the EAFs will be provided along with subsequent DAWN data releases. (*fig:EAFs*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2408.05275"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\orcid}[1]$
$\newcommand{\photoz}{photo-\textit{z}}$
$\newcommand{\specz}{spec-\textit{z}}$
$\newcommand{\Msol}{{\rm M}_\odot}$
$\newcommand{\classic}{\textsc{Classic}}$
$\newcommand{\lephare}{\texttt{LePhare}}$
$\newcommand{\eazy}{\texttt{EAZY}}$
$\newcommand{\tractor}{\texttt{The Tractor}}$
$\newcommand{\farmer}{\texttt{The Farmer}}$
$\newcommand{\sigmaz}{\sigma_{\Delta z / (1+z_{\rm s})}}$
$\newcommand{\zmin}{z_{\rm phot}^{\rm min}}$
$\newcommand{\zmax}{z_{\rm phot}^{\rm max}}$
$\newcommand{\SEP}{\texttt{SEP}}$
$\newcommand{\chOne}{[3.6\mum]}$
$\newcommand{\chTwo}{[4.5\mum]}$
$\newcommand{\hscpipe}{\texttt{hscpipe}}$</div>



<div id="title">

# $\Euclid$ preparation.

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2408.05296-b31b1b.svg)](https://arxiv.org/abs/2408.05296)<mark>Appeared on: 2024-08-13</mark> - 

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** The Cosmic Dawn Survey (DAWN survey) provides multiwavelength (UV/optical to mid-IR) data across the combined 59 deg $^{2}$ of the Euclid Deep and Auxiliary fields (EDFs and EAFs). Here, the first public data release (DR1) from the DAWN survey is presented. DR1 catalogues are made available for a subset of the full DAWN survey that consists of two Euclid Deep fields: Euclid Deep Field North (EDF-N) and Euclid Deep Field Fornax (EDF-F). The DAWN survey DR1 catalogues do not include $\Euclid$ data as they are not yet public for these fields. Nonetheless, each field has been covered by the ongoing Hawaii Twenty Square Degree Survey (H20), which includes imaging from CFHT MegaCam in the new $u$ filter and from Subaru Hyper Suprime-Cam (HSC) in the $griz$ filters. Each field is further covered by _Spitzer_ /IRAC 3.6-4.5 $\mu$ m imaging spanning 10 deg $^{2}$ and reaching $\sim$ 25 mag AB (5 $\sigma$ ). All present H20 imaging and all publicly available imaging from the aforementioned facilities are combined with the deep _Spitzer_ /IRAC data to create source catalogues spanning a total area of 16.87 deg $^{2}$ in EDF-N and 2.85 deg $^{2}$ in EDF-F for this first release. Photometry is measured from these multiwavelength data using $\farmer$ , a novel and well-validated model-based photometry code. Photometric redshifts and stellar masses are computed using two independent codes for modeling spectral energy distributions: $\eazy$ and $\lephare$ . Photometric redshifts show good agreement with spectroscopic redshifts ( $\sigma_{\rm NMAD} \sim  0.5, \eta < 8\%$ at $i < 25$ ). Number counts, photometric redshifts, and stellar masses are further validated in comparison to the COSMOS2020 catalogue. The DAWN survey DR1 catalogues are designed to be of immediate use in these two EDFs and will be continuously updated and made available as both new ground-based data and spaced-based data from $\Euclid$ are acquired and made public. Future data releases will provide catalogues of all EDFs and EAFs and include $\Euclid$ data.

</div>

<div id="div_fig1">

<img src="tmp_2408.05296/./figures/EDFN+F_mag_magErr.png" alt="Fig10" width="100%"/>

**Figure 10. -** Magnitude vs. magnitude error for each unique facility/filter combination, with EDF-N shown in colour and EDF-F shown in grey. Each solid line represents the median magnitude uncertainty as a function of magnitude for a given band. The shaded regions are bounded by the 84th and 16th percentiles of the magnitude error distributions, enclosing 68\% of the objects. The 5$\sigma$ limiting magnitudes measured with 2$\arcsec$  diameter apertures reported in Table \ref{tab:depths} are shown by vertical lines. A horizontal dotted line indicates a 5$\sigma$ model-based photometric uncertainty.  (*fig:mag_magerr*)

</div>
<div id="div_fig2">

<img src="tmp_2408.05296/./figures/fields_layout_in.png" alt="Fig6" width="100%"/>

**Figure 6. -** Layouts and facility coverage for the two fields included in the DAWN survey DR1 catalogues, EDF-N (left) and EDF-F (right). Also highlighted are areas of particular importance to \Euclid, namely, the \Euclid self-calibration field in EDF-N, and the Chandra Deep Field South (CDF-S) Euclid Auxiliary Field (EAF) in EDF-F. The regions spanned by the DR1 catalogues are illustrated by the green squares. The areas of the DR1 catalogues reaching approximately final integration times by all facilities (or covered to full depth; see Sect. \ref{subsec:area_coverage} and Table \ref{tab:exptimes}) are shown in grey. A future release from the DAWN survey will include catalogues with coverage from all facilities at full-depth spanning the entirety of EDF-N and EDF-F areas are targeted by \Euclid. (*fig:field_layouts*)

</div>
<div id="div_fig3">

<img src="tmp_2408.05296/./figures/EDFN_depthmaps_hex.png" alt="Fig8.1" width="50%"/><img src="tmp_2408.05296/./figures/EDFF_depthmaps_hex.png" alt="Fig8.2" width="50%"/>

**Figure 8. -** Depths and limiting magnitudes measured by the dispersion of empty aperture fluxes for each bandpass (see Sect. \ref{subsec:image_depths} and Appendix \ref{app:depths} for details). The top two rows correspond to EDF-N, and the bottom two rows correspond to EDF-F. The \enquote{full-depth} regions of EDF-N and EDF-F illustrated in Fig. \ref{fig:field_layouts} is apparent by the areas of greater depth in the HSC $griz$(see Sect. \ref{subsec:area_coverage}, Table \ref{tab:exptimes}, and Table \ref{tab:cat_areas}). The quoted value along the top of each panel is the median 5$\sigma$ depth the full-depth region of each  field. Only the areas covered by the DAWN survey DR1 catalogues is shown, corresponding to the green rectangular regions in Fig. \ref{fig:field_layouts}. Future data acquired from CFHT MegaCam and Subaru HSC will expand the areas of both fields and lead to more uniform coverage. The depths are summarized in Table \ref{tab:depths}. (*fig:depths*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2408.05296"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vsini}{V \sin i}$
$\newcommand{\vrad}{V_{\rm rad}}$
$\newcommand{\vmic}{V_{\rm mic}}$
$\newcommand{\vmac}{V_{\rm mac}}$
$\newcommand{\dex}{\rm dex}$
$\newcommand{\teff}{T_{\rm eff}}$
$\newcommand{\logg}{log {\it g_\star}}$
$\newcommand{\met}{[M/H]}$
$\newcommand{\feh}{[Fe/H]}$
$\newcommand{\cah}{[Ca/H]}$
$\newcommand{\sih}{[Si/H]}$
$\newcommand{\mgh}{[Mg/H]}$
$\newcommand{\nah}{[Na/H]}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\ms}{m~s^{-1}}$
$\newcommand{\msd}{m~s^{-1}~d^{-1}}$
$\newcommand{\gc}{g~cm^{-3}}$
$\newcommand{\lgr}{\log (R^\prime_{HK})}$
$\newcommand{\lgt}{\log~t}$
$\newcommand{\Lsun}{L_{\odot}}$
$\newcommand{\Msun}{M_{\odot}}$
$\newcommand{\Rsun}{R_{\odot}}$
$\newcommand{\mearth}{M_{\oplus}}$
$\newcommand{\MJ}{M_{Jup}}$
$\newcommand{\RJ}{R_{Jup}}$
$\newcommand{\rearth}{R_{\oplus}}$
$\newcommand{\rjup}{R_\mathrm{Jup}}$
$\newcommand{\mjup}{M_\mathrm{Jup}}$
$\newcommand{\Porb}{P_{orb}}$
$\newcommand{\Prot}{P_{rot}}$
$\newcommand{\mstar}{M_\star}$
$\newcommand{\rstar}{R_\star}$
$\newcommand{\lstar}{L_\star}$
$\newcommand{\rhostar}{\rho_{\mathrm{*}}}$
$\newcommand{\rhoplanet}{\rho_\mathrm{p}}$
$\newcommand{\rplanet}{R_{\mathrm{p}}}$
$\newcommand{\mplanet}{M_{\mathrm{p}}}$
$\newcommand{\Teq}{T_{eq}}$
$\newcommand{\toioneb}{TOI-2420 b}$
$\newcommand{\toitwob}{TOI-2485 b}$
$\newcommand{\pyaneti}{\href{https://github.com/oscaribv/pyaneti}{\texttt{pyaneti} \faGithub}}$
$\newcommand{\smass}[1][M_{\odot}]{1.1580000 _{- 0.0980000}^{ + 0.0980000}#1}$
$\newcommand{\sradius}[1][R_{\odot}]{ 2.3690000 _{ - 0.1240000}^{ + 0.1240000}#1}$
$\newcommand{\stemp}[1][\mathrm{K}]{5537.0000000 _{- 70.0000000}^{ + 70.0000000}#1 }$
$\newcommand{\Tzerob}[1][days]{8388.4119 \pm 0.0013 ~#1 }$
$\newcommand{\Pb}[1][days]{5.842641_{-0.000013}^{+0.000015} ~#1 }$
$\newcommand{\esinb}[1][ ]{0.05_{-0.20}^{+0.16} ~#1 }$
$\newcommand{\ecosb}[1][ ]{0.159_{-0.104}^{+0.074} ~#1 }$
$\newcommand{\bb}[1][ ]{0.849_{-0.025}^{+0.018} ~#1 }$
$\newcommand{\dentrheeb}[1][{\rm g^{1/3} cm^{-1}}]{0.133_{-0.026}^{+0.036} ~#1 }$
$\newcommand{\rrb}[1][ ]{0.05810_{-0.00083}^{+0.00098} ~#1 }$
$\newcommand{\kb}[1][{\rm m s^{-1}}]{94.25_{-6.29}^{+6.58} ~#1 }$
$\newcommand{\mpb}[1][M_\mathrm{J}]{0.927_{-0.079}^{+0.085} ~#1 }$
$\newcommand{\rpb}[1][R_\mathrm{J}]{1.340_{-0.072}^{+0.074} ~#1 }$
$\newcommand{\Tperib}[1][days]{8387.35_{-1.05}^{+0.74} ~#1 }$
$\newcommand{\eb}[1][ ]{0.055_{-0.031}^{+0.036} ~#1 }$
$\newcommand{\wb}[1][deg]{15.5_{-64.4}^{+47.5} ~#1 }$
$\newcommand{\prvb}[1][{\rm km s^{-1}}]{nan_{-nan}^{+nan} ~#1 }$
$\newcommand{\ib}[1][deg]{82.07_{-1.04}^{+0.98} ~#1 }$
$\newcommand{\arb}[1][ ]{6.21_{-0.43}^{+0.51} ~#1 }$
$\newcommand{\ab}[1][AU]{0.0684_{-0.0059}^{+0.0067} ~#1 }$
$\newcommand{\insolationb}[1][{\rm F_{\oplus}}]{1017_{-156}^{+167} ~#1 }$
$\newcommand{\tsmb}[1][ ]{13.78_{-1.53}^{+1.76} ~#1 }$
$\newcommand{\denstrb}[1][{\rm g cm^{-3}}]{0.133_{-0.026}^{+0.036} ~#1 }$
$\newcommand{\densspb}[1][{\rm g cm^{-3}}]{0.123_{-0.020}^{+0.024} ~#1 }$
$\newcommand{\Teqb}[1][K]{1571.6_{-64.0}^{+60.8} ~#1 }$
$\newcommand{\ttotb}[1][hours]{4.554_{-0.092}^{+0.093} ~#1 }$
$\newcommand{\tfulb}[1][hours]{2.93_{-0.15}^{+0.16} ~#1 }$
$\newcommand{\tegb}[1][hours]{0.81 \pm 0.11 ~#1 }$
$\newcommand{\deltamagb}[1][]{2.85_{-0.33}^{+0.28} ~#1 }$
$\newcommand{\denpb}[1][{\rm g cm^{-3}}]{0.477_{-0.080}^{+0.099} ~#1 }$
$\newcommand{\grapb}[1][{\rm cm s^{-2}}]{1346_{-193}^{+251} ~#1 }$
$\newcommand{\grapparsb}[1][{\rm cm s^{-2}}]{1283_{-167}^{+193} ~#1 }$
$\newcommand{\jspb}[1][ ]{94.78_{-9.95}^{+10.91} ~#1 }$
$\newcommand{\qoneSL}[1][]{0.096_{-0.063}^{+0.123} ~#1 }$
$\newcommand{\qtwoSL}[1][]{0.34_{-0.23}^{+0.35} ~#1 }$
$\newcommand{\uoneSL}[1][]{0.19_{-0.14}^{+0.20} ~#1 }$
$\newcommand{\utwoSL}[1][]{0.09_{-0.17}^{+0.21} ~#1 }$
$\newcommand{\qoneSC}[1][]{0.089_{-0.062}^{+0.145} ~#1 }$
$\newcommand{\qtwoSC}[1][]{0.22_{-0.16}^{+0.23} ~#1 }$
$\newcommand{\uoneSC}[1][]{0.119_{-0.088}^{+0.153} ~#1 }$
$\newcommand{\utwoSC}[1][]{0.15_{-0.13}^{+0.19} ~#1 }$
$\newcommand{\qonegblc}[1][]{0.80_{-0.18}^{+0.14} ~#1 }$
$\newcommand{\qtwogblc}[1][]{0.470_{-0.149}^{+0.096} ~#1 }$
$\newcommand{\uonegblc}[1][]{0.83_{-0.26}^{+0.12} ~#1 }$
$\newcommand{\utwogblc}[1][]{0.05_{-0.16}^{+0.27} ~#1 }$
$\newcommand{\CHIRON}[1][{\rm km s^{-1}}]{15.8940_{-0.0097}^{+0.0108} ~#1 }$
$\newcommand{\TULL}[1][{\rm km s^{-1}}]{10.2195_{-0.0059}^{+0.0060} ~#1 }$
$\newcommand{\Mthree}[1][{\rm km s^{-1}}]{16.5141_{-0.0094}^{+0.0092} ~#1 }$
$\newcommand{\Mfour}[1][{\rm km s^{-1}}]{16.502_{-0.017}^{+0.015} ~#1 }$
$\newcommand{\Mfive}[1][{\rm km s^{-1}}]{16.597 \pm 0.023 ~#1 }$
$\newcommand{\Msix}[1][{\rm km s^{-1}}]{16.552_{-0.013}^{+0.012} ~#1 }$
$\newcommand{\jCHIRON}[1][{\rm m s^{-1}}]{35.96_{-7.31}^{+10.31} ~#1 }$
$\newcommand{\jTULL}[1][{\rm m s^{-1}}]{10.06_{-8.17}^{+10.39} ~#1 }$
$\newcommand{\jMthree}[1][{\rm m s^{-1}}]{50.72_{-8.74}^{+10.42} ~#1 }$
$\newcommand{\jMfour}[1][{\rm m s^{-1}}]{22.5_{-20.3}^{+28.5} ~#1 }$
$\newcommand{\jMfive}[1][{\rm m s^{-1}}]{5.76_{-4.96}^{+26.22} ~#1 }$
$\newcommand{\jMsix}[1][{\rm m s^{-1}}]{50.0_{-13.1}^{+13.4} ~#1 }$
$\newcommand{\jtrSL}[1][ppm]{398.2_{-22.1}^{+27.5} ~#1 }$
$\newcommand{\jtrSC}[1][ppm]{858.8_{-25.1}^{+26.8} ~#1 }$
$\newcommand{\jtrgblc}[1][ppm]{2384.0_{-89.8}^{+91.5} ~#1 }$
$\newcommand{\Tzerobtwo}[1][days]{8939.7856_{-0.0023}^{+0.0022} ~#1 }$
$\newcommand{\Pbtwo}[1][days]{11.234790_{-0.000052}^{+0.000054} ~#1 }$
$\newcommand{\esinbtwo}[1][ ]{0.030_{-0.106}^{+0.091} ~#1 }$
$\newcommand{\ecosbtwo}[1][ ]{0.162_{-0.031}^{+0.023} ~#1 }$
$\newcommand{\bbtwo}[1][ ]{0.121_{-0.087}^{+0.134} ~#1 }$
$\newcommand{\dentrheebtwo}[1][{\rm g^{1/3} cm^{-1}}]{0.385_{-0.036}^{+0.031} ~#1 }$
$\newcommand{\rrbtwo}[1][ ]{0.06470_{-0.00068}^{+0.00067} ~#1 }$
$\newcommand{\kbtwo}[1][{\rm m s^{-1}}]{197.84_{-3.80}^{+3.99} ~#1 }$
$\newcommand{\mpbtwo}[1][M_\mathrm{J}]{2.412_{-0.087}^{+0.088} ~#1 }$
$\newcommand{\Tperibtwo}[1][days]{8937.40_{-1.13}^{+0.87} ~#1 }$
$\newcommand{\ebtwo}[1][ ]{0.0341_{-0.0087}^{+0.0109} ~#1 }$
$\newcommand{\wbtwo}[1][deg]{10.0_{-36.0}^{+27.7} ~#1 }$
$\newcommand{\prvbtwo}[1][{\rm km s^{-1}}]{nan_{-nan}^{+nan} ~#1 }$
$\newcommand{\ibtwo}[1][deg]{89.49_{-0.60}^{+0.36} ~#1 }$
$\newcommand{\arbtwo}[1][ ]{13.69_{-0.44}^{+0.36} ~#1 }$
$\newcommand{\abtwo}[1][AU]{0.1093_{-0.0057}^{+0.0055} ~#1 }$
$\newcommand{\insolationbtwo}[1][{\rm F_{\oplus}}]{275.6_{-23.5}^{+27.2} ~#1 }$
$\newcommand{\rpbtwo}[1][R_\mathrm{J}]{1.083 \pm 0.045 ~#1 }$
$\newcommand{\tsmbtwo}[1][ ]{3.83_{-0.24}^{+0.27} ~#1 }$
$\newcommand{\denstrbtwo}[1][{\rm g cm^{-3}}]{0.385_{-0.036}^{+0.031} ~#1 }$
$\newcommand{\densspbtwo}[1][{\rm g cm^{-3}}]{0.322_{-0.038}^{+0.045} ~#1 }$
$\newcommand{\Teqbtwo}[1][K]{1134.0_{-25.0}^{+27.0} ~#1 }$
$\newcommand{\ttotbtwo}[1][hours]{6.567 \pm 0.088 ~#1 }$
$\newcommand{\tfulbtwo}[1][hours]{5.743_{-0.087}^{+0.083} ~#1 }$
$\newcommand{\tegbtwo}[1][hours]{0.4057_{-0.0084}^{+0.0223} ~#1 }$
$\newcommand{\deltamagbtwo}[1][]{0.036_{-0.030}^{+0.115} ~#1 }$
$\newcommand{\denpbtwo}[1][{\rm g cm^{-3}}]{2.36_{-0.28}^{+0.33} ~#1 }$
$\newcommand{\grapbtwo}[1][{\rm cm s^{-2}}]{5749_{-390}^{+317} ~#1 }$
$\newcommand{\grapparsbtwo}[1][{\rm cm s^{-2}}]{5112_{-438}^{+489} ~#1 }$
$\newcommand{\jspbtwo}[1][ ]{421.9_{-24.5}^{+25.5} ~#1 }$
$\newcommand{\qoneSLtwo}[1][]{0.69_{-0.26}^{+0.21} ~#1 }$
$\newcommand{\qtwoSLtwo}[1][]{0.32_{-0.17}^{+0.23} ~#1 }$
$\newcommand{\uoneSLtwo}[1][]{0.53_{-0.25}^{+0.23} ~#1 }$
$\newcommand{\utwoSLtwo}[1][]{0.29_{-0.37}^{+0.33} ~#1 }$
$\newcommand{\qoneSCtwo}[1][]{0.73_{-0.28}^{+0.19} ~#1 }$
$\newcommand{\qtwoSCtwo}[1][]{0.18_{-0.12}^{+0.19} ~#1 }$
$\newcommand{\uoneSCtwo}[1][]{0.30_{-0.20}^{+0.26} ~#1 }$
$\newcommand{\utwoSCtwo}[1][]{0.53_{-0.35}^{+0.25} ~#1 }$
$\newcommand{\qonegblctwo}[1][]{0.148_{-0.064}^{+0.124} ~#1 }$
$\newcommand{\qtwogblctwo}[1][]{0.54_{-0.29}^{+0.30} ~#1 }$
$\newcommand{\uonegblctwo}[1][]{0.40_{-0.17}^{+0.14} ~#1 }$
$\newcommand{\utwogblctwo}[1][]{-0.03_{-0.17}^{+0.28} ~#1 }$
$\newcommand{\CHIRONtwo}[1][{\rm km s^{-1}}]{-27.563_{-0.050}^{+0.046} ~#1 }$
$\newcommand{\FEROStwo}[1][{\rm km s^{-1}}]{-26.148_{-0.047}^{+0.042} ~#1 }$
$\newcommand{\TREStwo}[1][{\rm km s^{-1}}]{0.371_{-0.049}^{+0.044} ~#1 }$
$\newcommand{\jCHIRONtwo}[1][{\rm m s^{-1}}]{14.7_{-11.9}^{+11.1} ~#1 }$
$\newcommand{\jFEROStwo}[1][{\rm m s^{-1}}]{3.14_{-2.54}^{+5.09} ~#1 }$
$\newcommand{\jTREStwo}[1][{\rm m s^{-1}}]{13.8_{-11.4}^{+12.5} ~#1 }$
$\newcommand{\jtrSLtwo}[1][ppm]{702.8_{-41.7}^{+50.4} ~#1 }$
$\newcommand{\jtrSCtwo}[1][ppm]{2123.4_{-56.5}^{+59.5} ~#1 }$
$\newcommand{\jtrgblctwo}[1][ppm]{1766.3_{-37.0}^{+39.1} ~#1 }$
$\newcommand{\ltrendtwo}[1][{\rm m s^{-1} d^{-1}}]{-0.77_{-0.16}^{+0.18} ~#1 }$
$\newcommand{\qtrendtwo}[1][{\rm m s^{-1} d^{-2}}]{0.26_{-0.12}^{+0.11} ~#1 }$</div>



<div id="title">

# Mass determination of two Jupiter-sized planets orbiting slightly evolved stars: TOI-2420 b and TOI-2485 b 

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2408.05612-b31b1b.svg)](https://arxiv.org/abs/2408.05612)<mark>Appeared on: 2024-08-13</mark> - 

</div>
<div id="authors">

I. Carleo, et al. -- incl., <mark>J. Eberhardt</mark>, <mark>T. Henning</mark>, <mark>T. Trifonov</mark>

</div>
<div id="abstract">

**Abstract:** Hot and warm Jupiters might have undergone the same formation and evolution path, but the two populations exhibit different distributions of orbital parameters, challenging our understanding on their actual origin. The present work, which is the results of our warm Jupiters survey  carried out with the CHIRON spectrograph within the KESPRINT collaboration, aims to address this challenge by studying two planets that could help bridge the gap between the two populations. We report the confirmation and mass determination of a hot Jupiter (orbital period shorter than 10 days), TOI-2420 b, and a warm Jupiter, TOI-2485 b. We performed a joint analysis using a wide variety of spectral and photometric data in order to characterize these planetary systems. We found that TOI-2420 b has an orbital period of P $_{\rm b}$ =5.8 days, a mass of M $_{\rm b}$ =0.9 M $_{\rm J}$ and a radius of R $_{\rm b}$ =1.3 R $_{\rm J}$ , with a planetary density of 0.477 $\gc$ ; while TOI-2485 b has an orbital period of P $_{\rm b}$ =11.2 days, a mass of M $_{\rm b}$ =2.4 M $_{\rm J}$ and a radius of R $_{\rm b}$ =1.1 R $_{\rm J}$ with density 2.36 $\gc$ . With current parameters, the migration history for TOI-2420 b and TOI-2485 b is unclear: the high-eccentricity migration scenarios cannot be ruled out, and TOI-2485 b's characteristics may rather support this scenario.

</div>

<div id="div_fig1">

<img src="tmp_2408.05612/./Figures/transits-toi2485.png" alt="Fig6.1" width="50%"/><img src="tmp_2408.05612/./Figures/toi2485_2pb_rv.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** *Top panel:* Phase-folded transit light curve $\toitwob$. Nominal TESS and LCO observations are shown in light grey. Solid coloured circles represent the binned data. Transit models are shown with a solid black line. *Bottom panel:* Phase-folded RV signal for $\toitwob$ following the subtraction of the systemic velocities. Orange circles, diamonds and squares show CHIRON, FEROS and TRES RV data, respectively.  (*fig:toi2485fits*)

</div>
<div id="div_fig2">

<img src="tmp_2408.05612/./Figures/Fig_TOI-2420_bootstrap_WASP_LC.png" alt="Fig1" width="100%"/>

**Figure 1. -** GLS periodogram of the WASP-South data for TOI-2420 from 2006 to 2011. There is a possible signal near 40 d, along with aliases from the yearly sampling. The dotted horizontal line  is the 1\%-likelihood false-alarm level. (*fig:wasp*)

</div>
<div id="div_fig3">

<img src="tmp_2408.05612/./Figures/Fig_TOI-2420_bootstrap.png" alt="Fig2.1" width="50%"/><img src="tmp_2408.05612/./Figures/Fig_TOI-2485_bootstrap.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** Periodograms of the RVs data for TOI-2420 (upper panel) and TOI-2485 (lower panel). The dotted horizontal line represent the 0.01\% false-alarm level, while the green vertical line is the maximum power, which corresponds to the planetary period.  (*fig:periodograms*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2408.05612"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\be}{\begin{equation}}$
$\newcommand{\ee}{\end{equation}}$
$\newcommand\bear{#1}\ear{\begin{align}#1\end{align}}$
$\newcommand{\nline}{\notag \\}$
$\newcommand{\f}{\frac}$
$\newcommand{\de}{\mathrm{d}}$
$\newcommand{\del}{\partial}$
$\newcommand{\half}{\frac{1}{2}}$
$\newcommand{\im}{\mathrm{i}}$
$\newcommand{\e}{\mathrm{e}}$
$\newcommand{\Msun}{\mathrm{M}_{\odot}}$
$\newcommand{\eqn}[1]{equation~(\ref{#1})}$
$\newcommand{\eqns}[2]{equations~(\ref{#1}) and~(\ref{#2})}$
$\newcommand{\secn}[1]{Section~\ref{#1}}$
$\newcommand{\appndx}[1]{Appendix~\ref{#1}}$
$\newcommand{\fig}[1]{Fig.~\ref{#1}}$
$\newcommand{\figs}[1]{Figs.~\ref{#1}}$
$\newcommand{\tab}[1]{Table~\ref{#1}}$
$\newcommand{\TRC}[1]{{\color{red}[{\bf TRC:} #1]}}$
$\newcommand{\BM}[1]{{\color{blue}[{\bf }#1]}}$
$\newcommand{\AP}[1]{\textbfit{\color{purple}[AP: #1]}}$
$\newcommand{\red}[1]{{\color{red} #1}}$
$\usepackage[T1]{fontenc}$
$\usepackage{amsmath}$
$\DeclareRobustCommand{\VAN}[3]{#2}$
$\let\VANthebibliography\thebibliography$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\usepackage{graphicx}$
$\usepackage{txfonts}$
$\begin{document}$
$   \title{Prospects of FRBs and Large Scale 21~cm Power Spectra in Constraining the Epoch of Reionization}$
$   \author{Barun Maity$
$          \inst{1}}$
$   \institute{Max-Planck-Institut für Astronomie, Königstuhl 17, D-69117 Heidelberg, Germany\               \email{maity@mpia.de}}$
$   \date{Received XXX; accepted XXX}$
$  \abstract{The Epoch of Reionization (EoR) is a crucial link to grasp the complete evolutionary history of the universe. Several attempts with a variety of observables have been utilized in the past to understand the thermal and ionization evolution of the Intergalactic Medium during EoR. In this study, we explore the simultaneous prospects of two important observables which are expected to be available in the near future, i.e. Dispersion Measure (DM) of high redshift FRBs and large scale 21 cm power spectra. For this purpose, we use an earlier developed explicitly photon conserving semi numerical model, \texttt{SCRIPT} including realistic recombination and radiative feedback effect. We check that DM evolution of 100 mock FRBs at high redshifts (7.0\le z\le15.0) is sufficient to recover the underlying reionization model, while 1000 FRB mocks at redshift range can constrain the reionization timeline within the percentage level uncertainties at 68\% confidence limit. Further, we study the effect of including large scale 21~cm power spectra (using only a single bin, k\sim0.14~h/\mathrm{cMpc}) at three redshifts along with FRB DM distribution. The joint exploration using these two observables can significantly improve the constraints on the various parameters (\lesssim 8\% uncertainties for reionization interval and midpoint at 95\% confidence) alleviating the degeneracies and can narrow down the thermal history of the universe by discarding some of the extreme heating models.}$
$   \keywords{intergalactic medium -- cosmology: theory – dark ages, reionization, first stars -- large-scale structure of Universe}$
$   \maketitle$
$\n\end{document}\end{equation}}$
$\newcommand{\ee}{\end{equation}}$
$\newcommand{\nline}{\notag \\}$
$\newcommand{\f}{\frac}$
$\newcommand{\de}{\mathrm{d}}$
$\newcommand{\del}{\partial}$
$\newcommand{\half}{\frac{1}{2}}$
$\newcommand{\im}{\mathrm{i}}$
$\newcommand{\e}{\mathrm{e}}$
$\newcommand{\Msun}{\mathrm{M}_{\odot}}$
$\newcommand{\eqn}[1]{equation~(\ref{#1})}$
$\newcommand{\eqns}[2]{equations~(\ref{#1}) and~(\ref{#2})}$
$\newcommand{\secn}[1]{Section~\ref{#1}}$
$\newcommand{\appndx}[1]{Appendix~\ref{#1}}$
$\newcommand{\fig}[1]{Fig.~\ref{#1}}$
$\newcommand{\figs}[1]{Figs.~\ref{#1}}$
$\newcommand{\tab}[1]{Table~\ref{#1}}$
$\newcommand{\TRC}[1]{{\color{red}[{\bf TRC:} #1]}}$
$\newcommand{\BM}[1]{{\color{blue}[{\bf }#1]}}$
$\newcommand{\AP}[1]{\textbfit{\color{purple}[AP: #1]}}$
$\newcommand{\red}[1]{{\color{red} #1}}$
$\newcommand{\arraystretch}{0.5}$
$\newcommand\bear{#1}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Prospects of FRBs and Large Scale 21 cm Power Spectra in Constraining the Epoch of Reionization

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2408.05722-b31b1b.svg)](https://arxiv.org/abs/2408.05722)<mark>Appeared on: 2024-08-13</mark> -  _Accepted for publication in A&A_

</div>
<div id="authors">

<mark>B. Maity</mark>

</div>
<div id="abstract">

**Abstract:** The Epoch of Reionization (EoR) is a crucial link to grasp the complete evolutionary history of the universe. Several attempts with a variety of observables have been utilized in the past to understand the thermal and ionization evolution of the Intergalactic Medium during EoR. In this study, we explore the simultaneous prospects of two important observables which are expected to be available in the near future, i.e. Dispersion Measure (DM) of high redshift FRBs and large scale 21 cm power spectra. For this purpose, we use an earlier developed explicitly photon conserving semi numerical model, \texttt{SCRIPT} including realistic recombination and radiative feedback effect. We check that DM evolution of 100 mock FRBs at high redshifts ( $7.0\le z\le15.0$ ) is sufficient to recover the underlying reionization model, while 1000 FRB mocks at redshift range can constrain the reionization timeline within the percentage level uncertainties at 68 \% confidence limit. Further, we study the effect of including large scale 21 cm power spectra (using only a single bin, $k\sim0.14 h/\mathrm{cMpc}$ ) at three redshifts along with FRB DM distribution. The joint exploration using these two observables can significantly improve the constraints on the various parameters ( $\lesssim 8\%$ uncertainties for reionization interval and midpoint at 95 \% confidence) alleviating the degeneracies and can narrow down the thermal history of the universe by discarding some of the extreme heating models.

</div>

<div id="div_fig1">

<img src="tmp_2408.05722/./params_corner_constr_models.png" alt="Fig3" width="100%"/>

**Figure 3. -** The comparison of parameter posteriors for three different sets of FRB mocks, i.e. with 100 (_black_), 500 (_green_) \& 1000 (_orange_) samples. The diagonal panels show the 1D posterior probability distribution while off diagonal panels show the joint 2D posteriors. The contours represent 68\% and 95\% confidence intervals. The dashed line represents the input parameter values used to generate the reionization model which has further been used to create the FRB mocks.  (*fig:corner_constr*)

</div>
<div id="div_fig2">

<img src="tmp_2408.05722/./DM_var_zeta0_rev1.png" alt="Fig1.1" width="25%"/><img src="tmp_2408.05722/./DM_var_alpha_rev1.png" alt="Fig1.2" width="25%"/><img src="tmp_2408.05722/./DM_var_Tre_rev1.png" alt="Fig1.3" width="25%"/><img src="tmp_2408.05722/./100_DM_mock_rev1.png" alt="Fig1.4" width="25%"/>

**Figure 1. -**  The first three panels (a,b,c) show the variation of globally averaged Dispersion Measure (DM) as a function of redshift for different free parameters in our model (i.e. $\zeta_0$, $\alpha$ and $T_{\mathrm{re}}$). We use $\zeta_0=12$, $\alpha=1.4$, and $T_{\mathrm{re}}=1.9\times10^4  \mathrm{K}$ as our fiducial model parameters in the analysis later. For each case, we show the plots for five different parameter values. The last panel (d) shows the fiducial mock DM values (at true redshifts) for 100 samples along with the corresponding input model. The mock data have been generated assuming a typical expectation from upcoming facilities like SKA. (*fig:DM_with_param*)

</div>
<div id="div_fig3">

<img src="tmp_2408.05722/./posterior_models_100_1000_FRBs_rev1.png" alt="Fig4" width="100%"/>

**Figure 4. -** Plots of 500 random models from the posterior chains for two different sets of FRB mocks (with 100 : _black_; with 1000 : _orange_). At the _left_ panel: we show the evolution of neutral hydrogen fraction with redshift. The red vertical dashed line represents our hard prior on the end of reionization at $z=5.3$. At the _right_ panel: we show the mean IGM temperature evolution. Although we don't use any other observational constraints while pursuing parameter space exploration, we show some of the existing observational estimates for ease of comparison.
    The magenta points at the _left_ represent the recent upper limits on neutral fraction from  ([Jin, Yang and Fan 2023]()) . The cyan points at the _right_ are the constraints on the mean IGM temperature using Ly-$\alpha$ forest spike statistics study  ([Gaikwad, Rauch and Haehnelt 2020]()) . The blue dashed line represents the input reionization model used for generating the FRB mocks.  (*fig:posterior_models*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2408.05722"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

160  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
